In [1]:
%matplotlib qt

from scipy.spatial.distance import pdist, squareform
from scipy.sparse.linalg import eigs, eigsh
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.lines import Line2D

In [2]:
# File Paths
data_path = "./data"
figure_path = "./plots"
abundance_table_path = f"{data_path}/abundance_table_97.shared"
metadata_path = f"{data_path}/SuperTransect_mapping_file.csv"

In [3]:
# Abundance Table
with open(abundance_table_path, "r") as file_literal:
    raw_abundance_data = [line.strip().split("\t") for line in file_literal]
    otu_names = raw_abundance_data[0][3:]
    sample_names = list(map(int, [line[1] for line in raw_abundance_data[1:]]))
    otu_counts = [line[3:] for line in raw_abundance_data[1:]]
abundance_table = pd.DataFrame(
    np.array(otu_counts, dtype=np.int64),
    index=sample_names,
    columns=otu_names)
abundance_table["Abundance"] = abundance_table.sum(axis=1)
abundance_table["Presence"] = abundance_table.drop("Abundance", axis=1).where(
    abundance_table == 0, 1).sum(axis=1)

# Metadata
metadata = pd.read_csv(metadata_path, index_col=0)

In [4]:
# Analysis function
def abundance_to_eigenvector(filtered_abundance_table, debug=False, pandas_mode=False):
    adjacency_matrix = squareform(pdist(filtered_abundance_table, metric="minkowski", p=1))
    kernel = np.exp(- (adjacency_matrix ** 2) / (3000**2))
    diagonal = np.diag(np.sum(kernel,axis=1))
    laplacian = diagonal - kernel
    eigenvalues, eigenvectors = eigs(laplacian, k=len(laplacian) - 1, M=diagonal)
    sample_eigens = zip(eigenvalues.real, eigenvectors.T, filtered_abundance_table.index)
    eigenvalues, eigenvectors, sample_ids = zip(*sorted(sample_eigens, key = lambda tup:tup[0]))

    if debug:
        print("Adjacency Matrix:\n", adjacency_matrix, "\n")
        print("Kernel:\n", kernel, "\n")
        print("Diagonal:\n", diagonal, "\n")
        print("Laplacian:\n", laplacian, "\n")
        print("Eigenvalues:\n", eigenvalues, "\n")
        print("Eigenvectors:\n", eigenvectors, "\n")
        print("Sample ID's:\n", sample_ids, "\n")
    
    if pandas_mode:
        return pd.DataFrame(eigenvectors, columns = filtered_abundance_table.index), filtered_abundance_table.index
    
    return eigenvectors, filtered_abundance_table.index, eigenvalues

In [10]:
# Eigenvector 3D function
def eigenvector_to_plot(eigenvectors, metadata, title, text = None, color_descriptor = "gps"):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    labels = eigenvectors.columns
    eigenvectors = eigenvectors.to_numpy()
    
    if text == None and color_descriptor == None:
        ax.scatter3D(eigenvectors[1], eigenvectors[2], eigenvectors[3])
    if text == "samples":
        for label, x, y, z in zip(labels, eigenvectors[1], eigenvectors[2], eigenvectors[3]):
            ax.text(x, y, z, label, None)
    if color_descriptor == "gps":
        location_number = metadata["lat"] + metadata["long"]
        color_number = (location_number - min(location_number)) / (max(location_number) - min(location_number))
        colors = [(0, color, 0) for color in color_number]
        for color, x, y, z in zip(colors, eigenvectors[1], eigenvectors[2], eigenvectors[3]):
            ax.scatter(x, y, z, color=color)
        custom_lines = [Line2D([0], [0], color=(0, 1, 0), lw=4),
                        Line2D([0], [0], color=(0, 0, 0), lw=4)]
        ax.legend(custom_lines, ['Higher Elevation', 'Lower Elevation'], loc ="center left", bbox_to_anchor=(-.1, 0))
    elif color_descriptor == "standard":
        for label, x, y, z in zip(labels, eigenvectors[1], eigenvectors[2], eigenvectors[3]):
            if metadata.loc[label]["host"] == "Animal" and metadata.loc[label]["habitat"] == "Marine":
                ax.scatter(x,y,z,c="C0", marker=".")
            if metadata.loc[label]["host"] == "Animal" and metadata.loc[label]["habitat"] == "Riverine":
                ax.scatter(x,y,z,c="C1", marker=".")
            if metadata.loc[label]["host"] == "Animal" and metadata.loc[label]["habitat"] == "Terrestrial":
                ax.scatter(x,y,z,c="C2", marker=".")
            if metadata.loc[label]["host"] == "Nonhost" and metadata.loc[label]["habitat"] == "Marine":
                ax.scatter(x,y,z,c="C0", marker="^")
            if metadata.loc[label]["host"] == "Nonhost" and metadata.loc[label]["habitat"] == "Riverine":
                ax.scatter(x,y,z,c="C1", marker="^")
            if metadata.loc[label]["host"] == "Nonhost" and metadata.loc[label]["habitat"] == "Terrestrial":
                ax.scatter(x,y,z,c="C2", marker="^")
            if metadata.loc[label]["host"] == "Plant" and metadata.loc[label]["habitat"] == "Marine":
                ax.scatter(x,y,z,c="C0", marker="s")
            if metadata.loc[label]["host"] == "Plant" and metadata.loc[label]["habitat"] == "Riverine":
                ax.scatter(x,y,z,c="C1", marker="s")
            if metadata.loc[label]["host"] == "Plant" and metadata.loc[label]["habitat"] == "Terrestrial":
                ax.scatter(x,y,z,c="C2", marker="s")
    elif color_descriptor == "sites":
        for label, x, y, z in zip(labels, eigenvectors[1], eigenvectors[2], eigenvectors[3]):
            if metadata.loc[label]["site_name"] == "TwinRocks":
                ax.scatter(x,y,z,c="C0")
            if metadata.loc[label]["site_name"] == "UppersBeach":
                ax.scatter(x,y,z,c="C1")
            if metadata.loc[label]["site_name"] == "WaimeaBay":
                ax.scatter(x,y,z,c="C2")
            if metadata.loc[label]["site_name"] == "ThreeTablesBeach":
                ax.scatter(x,y,z,c="C3")
            if metadata.loc[label]["site_name"] == "SharksCove":
                ax.scatter(x,y,z,c="C4")
        custom_lines = [Line2D([0], [0], color="C0", lw=4),
                        Line2D([0], [0], color="C1", lw=4),
                        Line2D([0], [0], color="C2", lw=4),
                        Line2D([0], [0], color="C3", lw=4),
                        Line2D([0], [0], color="C4", lw=4)]
        ax.legend(custom_lines, ['TwinRocks', 'UppersBeach', 'WaimeaBay', 'ThreeTablesBeach', 'SharksCove'], loc ="center left", bbox_to_anchor=(-.1, 0))
    elif color_descriptor == "coral_type":
        for label, x, y, z in zip(labels, eigenvectors[1], eigenvectors[2], eigenvectors[3]):
            if "CoralType1A" in metadata.loc[label]["collection_label"]:
                ax.scatter(x,y,z,c="C0")
            elif "CoralType1B" in metadata.loc[label]["collection_label"]:
                ax.scatter(x,y,z,c="C1")
            elif "CoralType1C" in metadata.loc[label]["collection_label"]:
                ax.scatter(x,y,z,c="C2")
            elif "CoralType2A" in metadata.loc[label]["collection_label"]:
                ax.scatter(x,y,z,c="C3")
            elif "CoralType2B" in metadata.loc[label]["collection_label"]:
                ax.scatter(x,y,z,c="C4")
            elif "CoralType2C" in metadata.loc[label]["collection_label"]:
                ax.scatter(x,y,z,c="C5")
            elif "CoralType3A" in metadata.loc[label]["collection_label"]:
                ax.scatter(x,y,z,c="C6")
            elif "CoralType3B" in metadata.loc[label]["collection_label"]:
                ax.scatter(x,y,z,c="C7")
            elif "CoralType3C" in metadata.loc[label]["collection_label"]:
                ax.scatter(x,y,z,c="C8")
            custom_lines = [Line2D([0], [0], color="C0", lw=4),
                        Line2D([0], [0], color="C1", lw=4),
                        Line2D([0], [0], color="C2", lw=4),
                        Line2D([0], [0], color="C3", lw=4),
                        Line2D([0], [0], color="C4", lw=4),
                        Line2D([0], [0], color="C5", lw=4),
                        Line2D([0], [0], color="C6", lw=4),
                        Line2D([0], [0], color="C7", lw=4),
                        Line2D([0], [0], color="C8", lw=4)]
        ax.legend(custom_lines, ["CoralType1A", "CoralType1B", "CoralType1C", "CoralType2A", "CoralType2B", "CoralType2C", "CoralType3A", "CoralType3B", "CoralType3C"], loc ="center left", bbox_to_anchor=(-.1, 0))
    elif color_descriptor == "coral_type_non_specific":
        for label, x, y, z in zip(labels, eigenvectors[1], eigenvectors[2], eigenvectors[3]):
            if "CoralType1" in metadata.loc[label]["collection_label"]:
                ax.scatter(x,y,z,c="C0")
            elif "CoralType2" in metadata.loc[label]["collection_label"]:
                ax.scatter(x,y,z,c="C1")
            elif "CoralType3" in metadata.loc[label]["collection_label"]:
                ax.scatter(x,y,z,c="C2")
            custom_lines = [Line2D([0], [0], color="C0", lw=4),
                        Line2D([0], [0], color="C1", lw=4),
                        Line2D([0], [0], color="C2", lw=4)]
        ax.legend(custom_lines, ["CoralType1", "CoralType2", "CoralType3"], loc ="center left", bbox_to_anchor=(-.1, 0))
    elif color_descriptor != None:
        for color, x, y, z in zip(color_descriptor, eigenvectors[1], eigenvectors[2], eigenvectors[3]):
            ax.scatter(x, y, z, c=color)
    plt.title(title)
    plt.show()
    plt.savefig(f"{figure_path}/{title}.png")

In [38]:
# Abundance & Presence Plots
def abundence_hist(abundance_table, key, filterer):
    filtered_metadata = metadata.loc[metadata[key] == filterer]
    filtered_abundance = abundance_table.filter(
            items=list(filtered_metadata.index), axis=0)
    filtered_metadata = filtered_metadata.loc[filtered_abundance.index]
    plt.figure()
    plt.xlabel("Abundance")
    plt.ylabel("Samples")
    plt.title(f"Abundance vs Samples {key.capitalize()} {filterer.capitalize()}")
    plt.hist(filtered_abundance["Abundance"], bins=100)
    plt.savefig(f"{figure_path}/abundance_hist_{key}_{filterer}.png")
    plt.show()

def presence_hist(abundance_table, key, filterer):
    filtered_metadata = metadata.loc[metadata[key] == filterer]
    filtered_abundance = abundance_table.filter(
            items=list(filtered_metadata.index), axis=0)
    filtered_metadata = filtered_metadata.loc[filtered_abundance.index]
    plt.figure()
    plt.xlabel("Presence")
    plt.ylabel("Samples")
    plt.title(f"Presence vs Samples {key.capitalize()} {filterer.capitalize()}")
    plt.hist(filtered_abundance["Presence"], bins=100)
    plt.savefig(f"{figure_path}/presence_hist_{key}_{filterer}.png")
    plt.show()

def presence_vs_abundance_scatter(abundance_table, key, filterer):
    filtered_metadata = metadata.loc[metadata[key] == filterer]
    filtered_abundance = abundance_table.filter(
            items=list(filtered_metadata.index), axis=0)
    filtered_metadata = filtered_metadata.loc[filtered_abundance.index]
    plt.figure()
    plt.xlabel("Presence")
    plt.ylabel("Abundance")
    plt.title(f"Presence vs Abundance {key.capitalize()} {filterer.capitalize()}")
    plt.scatter(x=filtered_abundance["Presence"], y=filtered_abundance["Abundance"])
    plt.savefig(f"{figure_path}/presence_vs_abundance_scatter_{key}_{filterer}.png")
    plt.show()

In [39]:
abundence_hist(abundance_table, "sample_type", "Mosquito")
presence_hist(abundance_table, "sample_type", "Mosquito")
presence_vs_abundance_scatter(abundance_table, "sample_type", "Mosqitio")

"""
abundence_hist(abundance_table, "sample_type", "Coral")
presence_hist(abundance_table, "sample_type", "Coral")
presence_vs_abundance_scatter(abundance_table, "sample_type", "Coral")
abundence_hist(abundance_table, "sample_type", "Drosophila")
presence_hist(abundance_table, "sample_type", "Drosophila")
presence_vs_abundance_scatter(abundance_table, "sample_type", "Drosophila")
"""

'\nabundence_hist(abundance_table, "sample_type", "Coral")\npresence_hist(abundance_table, "sample_type", "Coral")\npresence_vs_abundance_scatter(abundance_table, "sample_type", "Coral")\nabundence_hist(abundance_table, "sample_type", "Drosophila")\npresence_hist(abundance_table, "sample_type", "Drosophila")\npresence_vs_abundance_scatter(abundance_table, "sample_type", "Drosophila")\n'

In [37]:
key = "host"
filterer = "Animal"
filtered_metadata = metadata.loc[metadata[key] == filterer]
filtered_abundance = abundance_table.filter(
        items=list(filtered_metadata.index), axis=0)
filtered_metadata = filtered_metadata.loc[filtered_abundance.index]
filtered_abundance

,Otu00003,Otu00004,Otu00005,Otu00006,Otu00007,Otu00008,Otu00009,Otu00010,Otu00011,Otu00012,...,Otu25365,Otu25380,Otu25387,Otu25399,Otu25438,Otu25456,Otu25470,Otu25476,Abundance,Presence
105635,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3660,377
105631,4,0,0,0,0,0,12,0,0,0,...,0,0,0,3,0,0,0,0,3682,215
105629,0,0,0,0,0,0,0,0,7,0,...,0,0,0,0,0,0,0,0,3696,156
105617,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3694,182
105615,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3661,250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105246,2444,0,0,0,1201,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3697,19
105200,2072,0,0,0,1593,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3698,14
105153,1993,0,0,43,1499,0,20,0,0,0,...,0,0,0,0,0,0,0,0,3700,51
105132,3622,0,0,0,66,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3700,7


In [6]:
# Filtering function
def filtered_data(key, filterer, dropper = None):
    filtered_metadata = metadata.loc[metadata[key] == filterer]
    if dropper == None:
        filtered_abundance = abundance_table.filter(
            items=list(filtered_metadata.index), axis=0).drop(["Abundance", "Presence"],axis=1)
    if dropper != None:
        filtered_abundance = abundance_table.filter(
            items=list(filtered_metadata.index), axis=0).drop(["Abundance", "Presence"],axis=1).drop(dropper, axis=0)
    filtered_metadata = filtered_metadata.loc[filtered_abundance.index]
    return filtered_abundance, filtered_metadata

In [11]:
# Filtering Based on Drosophila
filtered_abundance, filtered_metadata  = filtered_data("host", "Animal")
#filtered_metadata.to_csv("./animal.csv")
eigenvectors, filtered_abundance_index = abundance_to_eigenvector(filtered_abundance, pandas_mode = True)
eigenvector_to_plot(eigenvectors, filtered_metadata, "Animal Generalized Eigenvectors")

In [7]:
# Filtering Based on Drosophila
filtered_abundance, filtered_metadata  = filtered_data("sample_type", "Drosophila")
eigenvectors, filtered_abundance_index = abundance_to_eigenvector(filtered_abundance, pandas_mode = True)
eigenvector_to_plot(eigenvectors, filtered_metadata, "Drosophila Generalized Eigenvectors")

/home/zjabbar/.local/lib/python3.8/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1267: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  warnings.warn("k >= N - 1 for N * N square matrix. "


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
# Filtering Based on Drosophila
filtered_abundance, filtered_metadata  = filtered_data("sample_type", "Drosophila")
eigenvectors, filtered_abundance_index = abundance_to_eigenvector(filtered_abundance, pandas_mode = True)
eigenvector_to_plot(eigenvectors, filtered_metadata, "Drosophila Generalized Eigenvectors")

In [7]:
# Filtering Based on Coral
filtered_abundance, filtered_metadata  = filtered_data("sample_type", "Coral")
eigenvectors, filtered_abundance_index = abundance_to_eigenvector(filtered_abundance, pandas_mode = True)
eigenvector_to_plot(eigenvectors, filtered_metadata, "Coral Generalized Eigenvectors", color_descriptor="coral_type_non_specific") # 105467, 105548; 105566

/home/zjabbar/.local/lib/python3.8/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1267: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  warnings.warn("k >= N - 1 for N * N square matrix. "


In [9]:
# Filtering Based on Mosquito
filtered_abundance, filtered_metadata  = filtered_data("sample_type", "Mosquito")
eigenvectors, filtered_abundance_index = abundance_to_eigenvector(filtered_abundance, pandas_mode = True)
eigenvector_to_plot(eigenvectors, filtered_metadata, "Mosquito Generalized Eigenvectors")

/home/zjabbar/.local/lib/python3.8/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1267: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  warnings.warn("k >= N - 1 for N * N square matrix. "


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
# Filtering Based on Mosquito
filtered_abundance, filtered_metadata  = filtered_data("sample_type", "Mosquito", dropper = [105279, 105525, 105502])
eigenvectors, filtered_abundance_index = abundance_to_eigenvector(filtered_abundance, pandas_mode = True)
eigenvector_to_plot(eigenvectors, filtered_metadata, "Mosquito Generalized Eigenvectors Dropped 3")

/home/zjabbar/.local/lib/python3.8/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1267: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  warnings.warn("k >= N - 1 for N * N square matrix. "


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
# Using All Data
filtered_abundance, filtered_metadata  = abundance_table, metadata
eigenvectors, filtered_abundance_index = abundance_to_eigenvector(filtered_abundance, pandas_mode = True)
eigenvector_to_plot(eigenvectors, filtered_metadata, "All Generalized Eigenvectors", color_descriptor="standard")

/home/zjabbar/.local/lib/python3.8/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1267: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  warnings.warn("k >= N - 1 for N * N square matrix. "


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# Eigenvalues
filtered_abundance, filtered_metadata  = filtered_data("host", "Animal")
eigenvectors, filtered_abundance_index, eigenvalues = abundance_to_eigenvector(filtered_abundance)
print(eigenvalues)
plt.figure()
plt.title = "Eigenvalues"
plt.plot(eigenvalues, linestyle="", marker=".")

(7.122827492742301e-17, 0.046944116676849966, 0.09231920771186891, 0.18020807529211777, 0.20869946874320458, 0.21922528976029013, 0.2264191845123947, 0.2552085875211022, 0.26208904209463413, 0.2653555555560466, 0.27512096631898186, 0.2886399693441487, 0.2981694290761307, 0.2988882139228235, 0.3147541773461532, 0.3322502975204712, 0.37661916908992743, 0.4096446983643368, 0.41772924092373626, 0.4217014217174079, 0.42874007661538915, 0.4495485520239819, 0.4495919809017021, 0.4844033501092248, 0.5338391298487144, 0.5508320854844418, 0.5538325866712612, 0.5583440176790819, 0.5619421551577596, 0.5753929949326224, 0.5856604786148532, 0.5924463009218793, 0.6175279677711993, 0.6191451873943254, 0.6256556608386664, 0.6339582868183785, 0.6356532633440748, 0.6444401572787455, 0.6485934803064693, 0.6524378700981794, 0.6628950333630925, 0.6651950786718133, 0.6704881955858996, 0.6862548193431917, 0.695733544970987, 0.7032147648153609, 0.7119702223524846, 0.7222170319390918, 0.7268390680556731, 0.7273

/home/zjabbar/.local/lib/python3.8/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1267: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  warnings.warn("k >= N - 1 for N * N square matrix. "


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …